## OpenAI에서 지원하는 Assistants API를 활용하여 PPT파일을 생성해보자!
- Assistants : 대화 및 파일 생성, 코드 명령 수행, 데이터 검색 및 추출 등 특정 task에 특화되어 학습된 자연어 기반 생성형 인공지능 API
- OpenAI의 일반적인 Chat API처럼 대화형 서비스도 제공하지만 특정 작업에 특화하여 지원하거나 자동화에 중점을 둠으로써 챗봇, 가상비서, 자동응답 시스템 개발 등 필요한 작업에 포커스를 맞춰 활용 가능

### OpenAI API Key 설정

In [1]:
from openai import OpenAI
from getpass import getpass
import time

In [4]:
MY_API_KEY = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [6]:
client = OpenAI(api_key=MY_API_KEY)

### 2. 어시스턴트, 스레드, 메세지 갱체 생성
- 스레드(thread) : 프로세스 내에서 실행되는 가장 작은 실행 단위로 현재 실습에서는 어시스턴트와 사용자간 채팅 세션(꽁간)을 하나의 스레드로 지정하여 진행
- 메세지(message) : 스레드 내의 통신 단위

1. 어시스턴트 세부사항 정의
2. 어시스턴트 객체 생성
3. 스레드 및 메세지 객체 생성
4. 스레드 실행(대화 시작 및 요구사항 요청)

### 1) 어시스턴트 세부사항 정의

In [10]:
# 어시스턴트의 역할을 정의(role에서 system과 유사한 기능)
assistant_instruction = """등산에 관련된 PowerPoint 파일을 만들어야 해. 너는 등산 전문가이자 PowerPoint 전문가야.
전체적인 PPT의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해줘.
페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point 정도로 설정해줘.
슬라이드는 3개 만들어줘."""

# 원하는 요청을 작성
prompt_user = """입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘.
초급,중급,고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 포함하는 프레젠테이션을 만들어줘.
페이지 구성이 깔끔하고 내용은 구체적으로 작성해주면 좋겠어. 바로 작성해줘 화이팅~!"""

### 2) 어시스턴트 객체 생성

In [13]:
assistant = client.beta.assistants.create(name = "My assistant",
                                           instructions = assistant_instruction,
                                           # 현재 tools에 사용 가능한 타입은 code_interpreter, function, file_search 등이 있음
                                           tools = [{"type":"code_interpreter"}],
                                           model = "gpt-4o"
                                          )

# code_interpreter : 모델이 코드를 실행하여 계산, 데이터분석 파일 조작 등의 작업을 수행
# function : 특정 작업을 구현할 수 있는 사용자정의함수를 호출하여 작업 수행
# file_search : 사전 구성된 환경에서 파일을 검색하고 가져와 특정 작업 수행

In [14]:
assistant

Assistant(id='asst_fIG5ktJxitsxxFZCCUb8xcpY', created_at=1738636378, description=None, instructions='등산에 관련된 PowerPoint 파일을 만들어야 해. 너는 등산 전문가이자 PowerPoint 전문가야.\n전체적인 PPT의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해줘.\n페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point 정도로 설정해줘.\n슬라이드는 3개 만들어줘.', metadata={}, model='gpt-4o', name='My assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)

### 3) 스레드 및 메세지 객체 생성

In [18]:
# 스레드 객체 생성
# 대화를 시작하면 스레드가 생성되며 대하 중 교환되는 모든 메세지는 동일한 스레드에서 진행됨
thread = client.beta.threads.create()

#메세지 객체 생성
 # 메세지는 상ㅇ자의 질의, 명령과 같은 텍스트 뿐만 아니라 파일 등 모든 컨텐츠가 포함
 # 메세지는 스레드 내의 통신 단위이므로 위에서 생성한 스레드 id값을 입력
message = client.beta.threads.messages.create(thread_id = thread.id,
                                              role = "user",
                                              content=prompt_user
                                             )

In [19]:
print(thread)

Thread(id='thread_jtIttaYLFuxq4BEtc3ei2bmN', created_at=1738636379, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [22]:
print(message)

Message(id='msg_5hqqyzklQi3aI0YikzRTL56p', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘.\n초급,중급,고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 포함하는 프레젠테이션을 만들어줘.\n페이지 구성이 깔끔하고 내용은 구체적으로 작성해주면 좋겠어. 바로 작성해줘 화이팅~!'), type='text')], created_at=1738636379, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_jtIttaYLFuxq4BEtc3ei2bmN')


### 4) 스레드 실행(때화 시작 및 요구사항 요청)
- 위에 설정된 정보로 역할 설정 및 원하는 작업 요청하는 단계

In [33]:
# 스레드 실행(실행하면 llM으로 사용자의 요청이 넘어감)
# 스레드는 사용자가 요청을 입력했을 때 시작되고 모델의 응답이 끝나면 종료됨
run = client.beta.threads.runs.create(thread_id = thread.id,
                                      assistant_id = assistant.id
                                     )

In [36]:
run

Run(id='run_IBiFvwPGo2DfLyGMGNfTfUgM', assistant_id='asst_fIG5ktJxitsxxFZCCUb8xcpY', cancelled_at=None, completed_at=None, created_at=1738636821, expires_at=1738637421, failed_at=None, incomplete_details=None, instructions='등산에 관련된 PowerPoint 파일을 만들어야 해. 너는 등산 전문가이자 PowerPoint 전문가야.\n전체적인 PPT의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해줘.\n페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point 정도로 설정해줘.\n슬라이드는 3개 만들어줘.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_jtIttaYLFuxq4BEtc3ei2bmN', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

- 스레드를 실행시킨다고 설정해 둔 모든 요청과 응답이 즉각적으로 이루어지지는 않음
- 생성 모델이 사용자의 요청을 이닞하고 결과물을 생성하기 위해서는 어느 정도의 시간이 필요함(복잡하고 어려운 요청일수록 더 시간이 오래 걸릴 수 있음)

###ASSISTANTS 응답 진행 상황 확인 코드###
- 작업을 완료하는 데 충분한 시간이 흘렀다면 실행 완료가 뜨고 실행중이라면 실행되고 있는 과정이 체크되도록 코드 작성

In [39]:
while True:
    # retrieve : 특정 스레드의 실행 상태나 실행 후 결과를 검색하는데 활용하는 함수
    run_retrieve = client.beta.threads.runs.retrieve(thread_id = thread.id,
                                                    run_id = run.id
                                                   )
    # 실행 완료인 경우
     # status는 inprogress, completed, failed로 출력될 수 있음
    if run_retrieve.status == "completed":
        print("작업 완료!")
        break
    # 작업이 실행중 또는 실패인 경우
    else:
        print(run_retrieve.status)
        time.sleep(3)

in_progress
작업 완료!


In [41]:
# messages.list : 특정 스레드의 메세지에 대한 정보와 그 목록을 보여주는 함수
messages = client.beta.threads.messages.list(thread_id = thread.id)
messages

# 맨 아래부터 사용자의 요청 메세지와 그에 대한 모델의 응답 메세지가 순차적으로 출력되고 있음

SyncCursorPage[Message](data=[Message(id='msg_joDl6ERETBqAPtYuu6VJCTiY', assistant_id='asst_fIG5ktJxitsxxFZCCUb8xcpY', attachments=[Attachment(file_id='file-JmFyxEmTC5qtLpbqkq7Dzm', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=116, file_path=FilePath(file_id='file-JmFyxEmTC5qtLpbqkq7Dzm'), start_index=74, text='sandbox:/mnt/data/Hiking_Presentation.pptx', type='file_path')], value='프레젠테이션 자료가 완성되었습니다. 아래 링크를 통해 다운로드하실 수 있습니다:\n\n[등산 입문자를 위한 프레젠테이션 자료 다운로드](sandbox:/mnt/data/Hiking_Presentation.pptx) \n\n필요한 경우 내용 추가나 수정 요청해 주세요!'), type='text')], created_at=1738636822, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_IBiFvwPGo2DfLyGMGNfTfUgM', status=None, thread_id='thread_jtIttaYLFuxq4BEtc3ei2bmN'), Message(id='msg_Yer5SgD3G7oBKVMtsEdxMR2z', assistant_id='asst_fIG5ktJxitsxxFZCCUb8xcpY', attachments=[Attachment(file

In [49]:
messages.data[-2]

Message(id='msg_12cBYQYJdCOl5IEveyHJ0Xgr', assistant_id='asst_fIG5ktJxitsxxFZCCUb8xcpY', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='아래의 내용을 포함해서 PowerPoint 프레젠테이션을 제작해 보겠습니다. 각 슬라이드의 주제와 세부 내용을 다음과 같이 구성하려고 합니다.\n\n### 슬라이드 1: 등산 개요\n- **제목**: 등산 개요\n- **내용**:\n  - 등산의 장점\n  - 자연 속에서의 건강 유지\n  - 정신적 및 신체적 도전 제공\n\n### 슬라이드 2: 수준별 등산 계획\n- **제목**: 수준별 등산 계획\n- **내용**:\n  - 초급: 매주 1회, 산 높이 500m 이하\n  - 중급: 매주 1-2회, 산 높이 500-1,000m\n  - 고급: 매주 2회 이상, 산 높이 1,000m 이상\n  - 권장 스트레칭:\n    - 다리 근육 풀기\n    - 어깨와 허리 유연성 향상\n\n### 슬라이드 3: 등산화 및 장비 추천\n- **제목**: 등산화 및 장비 추천\n- **내용**:\n  - 초급: 가벼운 트레킹화\n  - 중급: 견고한 등산화\n  - 고급: 방수 및 강한 내구성 등산화\n  - 추가 장비:\n    - 등산용 스틱, 보호용 양말, 모자 및 자외선 차단제\n\n이 구성을 바탕으로 PPT를 제작하여 공유해드리겠습니다.'), type='text')], created_at=1738636388, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TT7X2Zn0Ivryuy0smhSVvRUH', status=None, thread_id='thread_jtIttaYLFuxq4BE

In [51]:
messages.data[0].content[0].text

Text(annotations=[FilePathAnnotation(end_index=116, file_path=FilePath(file_id='file-JmFyxEmTC5qtLpbqkq7Dzm'), start_index=74, text='sandbox:/mnt/data/Hiking_Presentation.pptx', type='file_path')], value='프레젠테이션 자료가 완성되었습니다. 아래 링크를 통해 다운로드하실 수 있습니다:\n\n[등산 입문자를 위한 프레젠테이션 자료 다운로드](sandbox:/mnt/data/Hiking_Presentation.pptx) \n\n필요한 경우 내용 추가나 수정 요청해 주세요!')

In [53]:
# annotations : 메세지 유형(텍스트, 이미지, 비디오, 각종파일)에 대한 정보가 담겨져 있음
messages.data[0].content[0].text.annotations

[FilePathAnnotation(end_index=116, file_path=FilePath(file_id='file-JmFyxEmTC5qtLpbqkq7Dzm'), start_index=74, text='sandbox:/mnt/data/Hiking_Presentation.pptx', type='file_path')]

### annotations가 비어있는 경우의 원인
 - 생성 모델이 결과물을 전부 생성하기 전에 코드를 실행한경우
 - 필요한 데이터가 메세지에 포함되지 않았을 경우(파일 미첨부, 애매한 지시 등)
 - API호출에 문제가 있거나 잘못된 thread_id를 사용하여 올바른 데이터를 가져오지 못한 경우

In [67]:
# 생성한 파일의 정보 추출

# 메세지를 통해 받은 정보 중 생성한 파일(PPT)의 file_id 값을 변수에 저장
file_id_path = messages.data[0].content[0].text.annotations[0].file_path.file_id

# retrieve_content : file_id 값을 통해 생성된 파일의 내용들을 검색해 가져오는 함수
file_contents = client.files.with_raw_response.retrieve_content(file_id_path)

In [69]:
# result 폴더에 파일 저장
with open("result/PPT_001.pptx", "wb") as f :
    # 메세지에서 가져온 파일의 내용을 바탕으로 실제 생성할 파일 컨텐츠 기기
    f.write(file_contents.content)

### 생성 파일 정보 및 어시스턴트 삭제(메모리 끊기)

In [78]:
client.files.delete(file_id_path)
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_fIG5ktJxitsxxFZCCUb8xcpY', deleted=True, object='assistant.deleted')

### 정리
- 간단한 요청이었기에 결과물의 퀄리티가 높지는 않았찌만 우리가 요청한 내용들이 잘 반영되어 있음
- assistants API를 사용하면 일반 대화형 모델로 하기 힘든 파일 제작 등과 같은 특정 task의 결과물을 얻을 수 있음